In [7]:
class SparseMatrix:
    def __init__(self, shape: tuple[int, int], dtype=float):
        self.shape = shape
        self.rows, self.cols = shape
        self.dtype = dtype
        self.format = None



class CSCMatrix(SparseMatrix):
    def __init__(self, shape: tuple[int,int] = (0,0), dense_matrix=None):
        if dense_matrix is not None:
            # Extract shape and dtype from dense_matrix
            shape = (len(dense_matrix), len(dense_matrix[0]))
            dtype = type(dense_matrix[0][0])
            super().__init__(shape, dtype)   # Initialize parent
        else:
            super().__init__(shape)           # Initialize parent with given shape

        # Now set CSC-specific attributes
        self.values = []
        self.row_indices = []
        self.col_ptr = [0]
        self.format = 'CSC'

        # If dense_matrix provided, convert to CSC
        if dense_matrix is not None:
            self._from_dense(dense_matrix)   # Note: single underscore

    def _from_dense(self,dense_matrix):
        """Convert dense matrix to CSC format"""
        for j in range(self.cols):
            col_nonzero = 0
            for i in range(self.rows):
                if dense_matrix[i][j] != 0:
                    self.values.append(dense_matrix[i][j])
                    self.row_indices.append(i)
                    col_nonzero += 1
            self.col_ptr.append(self.col_ptr[-1] + col_nonzero)




dense_matrix = [
        [1, 0, 0, 2],
        [0, 0, 3, 0],
        [0, 4, 0, 5],
        [6, 0, 0, 7]
    ]
    

csc_matrix= CSCMatrix(dense_matrix= dense_matrix)



In [5]:
class COOMatrix:
    def __init__(self, dense_matrix):
        self.shape = (len(dense_matrix), len(dense_matrix[0]))
        self.data= []
        self.row = []
        self.col = []


matrix = [[1,2], [3,4]]
temp= COOMatrix(matrix)

temp.shape

(2, 2)